# 听牌分析与手牌规范化

演示 holdem-lab 新增的两个功能模块：
- **draws.py** - 听牌分析（Outs 计算）
- **canonize.py** - 手牌规范化（169 种起手牌）

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    # 听牌分析
    DrawType, FlushDraw, StraightDraw, DrawAnalysis,
    analyze_draws, count_flush_outs, count_straight_outs, get_primary_draw,
    # 手牌规范化
    CanonicalHand, canonize_hole_cards, parse_canonical_hand,
    get_all_combos, get_all_canonical_hands, get_combos_excluding,
)
print("模块导入成功！")

---
## 第一部分：手牌规范化 (Canonize)

将具体手牌映射到 169 种规范起手牌类型。

### 1.1 基本规范化

In [ ]:
# 不同花色的 AK suited 都会规范化为同一个 CanonicalHand
hands = [
    parse_cards("Ah Kh"),  # 红桃 AK
    parse_cards("As Ks"),  # 黑桃 AK
    parse_cards("Ad Kd"),  # 方块 AK
    parse_cards("Ac Kc"),  # 梅花 AK
]

print("同花 AK 的规范化结果：")
for h in hands:
    canonical = canonize_hole_cards(h)
    print(f"  {format_cards(h)} -> {canonical}")

# 验证它们都相等
canonicals = [canonize_hole_cards(h) for h in hands]
print(f"\n所有结果相等: {len(set(canonicals)) == 1}")

In [ ]:
# 同花 vs 杂花
suited = parse_cards("Ah Kh")
offsuit = parse_cards("Ah Ks")

print(f"同花:  {format_cards(suited)} -> {canonize_hole_cards(suited)}")
print(f"杂花:  {format_cards(offsuit)} -> {canonize_hole_cards(offsuit)}")

In [ ]:
# 对子
pocket_aces = parse_cards("Ah Ad")
pocket_kings = parse_cards("Kh Kd")
pocket_deuces = parse_cards("2h 2d")

print("对子的规范化：")
print(f"  {format_cards(pocket_aces)} -> {canonize_hole_cards(pocket_aces)}")
print(f"  {format_cards(pocket_kings)} -> {canonize_hole_cards(pocket_kings)}")
print(f"  {format_cards(pocket_deuces)} -> {canonize_hole_cards(pocket_deuces)}")

### 1.2 解析规范记号

In [ ]:
# 从字符串解析
hands_to_parse = ["AA", "AKs", "AKo", "72o", "JTs", "QQ"]

print("解析规范记号：")
print("-" * 40)
for s in hands_to_parse:
    h = parse_canonical_hand(s)
    print(f"  {s:5} -> {h!r}")
    print(f"         组合数: {h.num_combos}, 是对子: {h.is_pair}, gap: {h.gap}")

### 1.3 展开为具体组合

In [ ]:
# AKs 有 4 种具体组合
aks = parse_canonical_hand("AKs")
combos = get_all_combos(aks)

print(f"{aks} 的所有组合 ({len(combos)} 种):")
for c1, c2 in combos:
    print(f"  {c1.pretty()}{c2.pretty()}")

In [ ]:
# AA 有 6 种具体组合 (C(4,2) = 6)
aa = parse_canonical_hand("AA")
combos = get_all_combos(aa)

print(f"{aa} 的所有组合 ({len(combos)} 种):")
for c1, c2 in combos:
    print(f"  {c1.pretty()}{c2.pretty()}")

In [ ]:
# 排除已知死牌后的组合
aa = parse_canonical_hand("AA")
dead = parse_cards("Ah Kc")  # 假设 Ah 已被其他玩家持有

remaining = get_combos_excluding(aa, dead)
print(f"{aa} 排除 {format_cards(dead)} 后的组合 ({len(remaining)} 种):")
for c1, c2 in remaining:
    print(f"  {c1.pretty()}{c2.pretty()}")

### 1.4 所有 169 种起手牌

In [ ]:
all_hands = get_all_canonical_hands()

# 统计
pairs = [h for h in all_hands if h.is_pair]
suited = [h for h in all_hands if h.suited]
offsuit = [h for h in all_hands if not h.suited and not h.is_pair]

print(f"总起手牌类型: {len(all_hands)}")
print(f"  对子: {len(pairs)} 种")
print(f"  同花: {len(suited)} 种")
print(f"  杂花: {len(offsuit)} 种")

# 验证总组合数
total_combos = sum(h.num_combos for h in all_hands)
print(f"\n总组合数: {total_combos} (应该等于 C(52,2) = 1326)")

In [ ]:
# 以 13x13 矩阵形式显示
from holdem_lab import Rank

print("169 起手牌矩阵 (对角线=对子, 上三角=同花, 下三角=杂花):")
print()

ranks = list(reversed(Rank))  # A, K, Q, ..., 2
print("    ", end="")
for r in ranks:
    print(f"{r:>4}", end="")
print()

for i, r1 in enumerate(ranks):
    print(f"{r1:>3} ", end="")
    for j, r2 in enumerate(ranks):
        if i == j:
            # 对子
            print(f"{r1}{r2}  ", end="")
        elif i < j:
            # 同花 (上三角)
            print(f"{r1}{r2}s ", end="")
        else:
            # 杂花 (下三角)
            print(f"{r2}{r1}o ", end="")
    print()

---
## 第二部分：听牌分析 (Draws)

分析手牌的听牌情况和补牌数 (Outs)。

### 2.1 同花听牌 (Flush Draw)

In [ ]:
# 经典同花听牌: 4 张同花
hole = parse_cards("Ah Kh")
board = parse_cards("7h 6h 2c")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print()

if analysis.flush_draws:
    fd = analysis.flush_draws[0]
    print(f"同花听牌: {fd.suit.name}")
    print(f"  持有张数: {fd.cards_held}")
    print(f"  补牌数: {fd.out_count}")
    print(f"  是坚果: {fd.is_nut}")
    print(f"  具体补牌: {' '.join(c.pretty() for c in fd.outs[:5])}...")

In [ ]:
# 非坚果同花听牌
hole = parse_cards("Kh 2h")
board = parse_cards("7h 6h 3c")

analysis = analyze_draws(hole, board)
fd = analysis.flush_draws[0]

print(f"手牌: {format_cards(hole)}")
print(f"是坚果同花听牌: {fd.is_nut}  (因为没有 A♥)")

### 2.2 顺子听牌 (Straight Draw)

In [ ]:
# 两头顺 (Open-Ended Straight Draw, OESD)
hole = parse_cards("9h 8c")
board = parse_cards("7d 6s 2h")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print(f"持有: 6-7-8-9")
print()

for sd in analysis.straight_draws:
    print(f"顺子听牌: {sd.draw_type.name}")
    print(f"  需要的点数: {sd.needed_ranks}")
    print(f"  补牌数: {sd.out_count}")
    print(f"  完成后最高牌: {sd.high_card}")

In [ ]:
# 卡顺 (Gutshot)
hole = parse_cards("9h 6c")
board = parse_cards("8d 5s 2h")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print(f"持有: 5-6-8-9 (缺 7)")
print()

for sd in analysis.straight_draws:
    if sd.draw_type == DrawType.GUTSHOT:
        print(f"卡顺: 需要 {sd.needed_ranks[0]}")
        print(f"补牌数: {sd.out_count}")

In [ ]:
# Wheel 听牌 (A-2-3-4-5)
hole = parse_cards("Ah 4c")
board = parse_cards("3d 2s Kh")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print(f"持有: A-2-3-4 (wheel 听牌)")
print()

for sd in analysis.straight_draws:
    print(f"听牌类型: {sd.draw_type.name}")
    print(f"需要: {sd.needed_ranks}")
    print(f"完成后是 {sd.high_card}-high 顺子")

### 2.3 组合听牌 (Combo Draw)

In [ ]:
# 怪兽听牌: 同花听牌 + 两头顺
hole = parse_cards("9h 8h")
board = parse_cards("7h 6c 2h")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print()
print(f"是组合听牌: {analysis.is_combo_draw}")
print()

# 同花听牌
if analysis.flush_draws:
    fd = analysis.flush_draws[0]
    print(f"同花听牌: {fd.out_count} outs")

# 顺子听牌
if analysis.straight_draws:
    sd = analysis.straight_draws[0]
    print(f"顺子听牌 ({sd.draw_type.name}): {sd.out_count} outs")

# 总补牌数 (去重)
print(f"\n总补牌数 (去重后): {analysis.total_outs}")
print(f"注: 同时完成同花和顺子的牌只计一次")

In [ ]:
# 显示所有补牌
print("所有补牌 (去重):")
for i, out in enumerate(analysis.all_outs):
    print(f"  {out.pretty()}", end="")
    if (i + 1) % 10 == 0:
        print()

### 2.4 已成牌的情况

In [ ]:
# 已经成同花
hole = parse_cards("Ah Kh")
board = parse_cards("Qh Jh 2h")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print()
print(f"已成同花: {analysis.has_flush}")
print(f"同花听牌数: {len(analysis.flush_draws)}  (已成牌不算听牌)")

In [ ]:
# 已经成顺子
hole = parse_cards("9h 8c")
board = parse_cards("7d 6s 5h")

analysis = analyze_draws(hole, board)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print()
print(f"已成顺子: {analysis.has_straight}")
print(f"顺子听牌数: {len(analysis.straight_draws)}  (已成牌不算听牌)")

### 2.5 便捷函数

In [ ]:
# 快速计算补牌数
scenarios = [
    ("Ah Kh", "7h 6h 2c", "同花听牌"),
    ("9h 8c", "7d 6s 2h", "两头顺"),
    ("9h 6c", "8d 5s 2h", "卡顺"),
    ("9h 8h", "7h 6c 2h", "组合听牌"),
]

print("快速补牌统计:")
print("-" * 60)
print(f"{'场景':12} {'手牌':10} {'公共牌':14} {'同花':6} {'顺子':6}")
print("-" * 60)

for hole_str, board_str, desc in scenarios:
    hole = parse_cards(hole_str)
    board = parse_cards(board_str)
    
    flush = count_flush_outs(hole, board)
    straight = count_straight_outs(hole, board)
    
    print(f"{desc:12} {hole_str:10} {board_str:14} {flush:6} {straight:6}")

In [ ]:
# 获取主要听牌类型
scenarios = [
    ("Ah Kh", "7h 6h 2c"),
    ("9h 8c", "7d 6s 2h"),
    ("Ah Kc", "7d 6s 2h"),  # 无听牌
]

print("主要听牌类型:")
for hole_str, board_str in scenarios:
    hole = parse_cards(hole_str)
    board = parse_cards(board_str)
    
    primary = get_primary_draw(hole, board)
    primary_str = primary.name if primary else "无"
    
    print(f"  {hole_str} + {board_str} -> {primary_str}")

### 2.6 死牌影响

In [ ]:
# 同花听牌，但部分补牌已被对手持有
hole = parse_cards("Ah Kh")
board = parse_cards("7h 6h 2c")
dead = parse_cards("Qh Jh Th")  # 对手持有 3 张红桃

# 无死牌
analysis_clean = analyze_draws(hole, board)
# 有死牌
analysis_dead = analyze_draws(hole, board, dead_cards=dead)

print(f"手牌: {format_cards(hole)}")
print(f"公共牌: {format_cards(board)}")
print(f"已知死牌: {format_cards(dead)}")
print()
print(f"无死牌时补牌数: {analysis_clean.flush_draws[0].out_count}")
print(f"有死牌时补牌数: {analysis_dead.flush_draws[0].out_count}")

---
## 总结

| 模块 | 功能 | 主要函数 |
|------|------|----------|
| canonize | 手牌规范化 | `canonize_hole_cards()`, `parse_canonical_hand()`, `get_all_combos()` |
| draws | 听牌分析 | `analyze_draws()`, `count_flush_outs()`, `count_straight_outs()` |

### 听牌类型

| 类型 | 英文 | 补牌数 |
|------|------|--------|
| 同花听牌 | Flush Draw | 9 |
| 两头顺 | Open-Ended (OESD) | 8 |
| 双卡顺 | Double Gutshot | 8 |
| 卡顺 | Gutshot | 4 |
| 后门同花 | Backdoor Flush | ~1 (需两张) |
| 后门顺子 | Backdoor Straight | ~1 (需两张) |